### Fun with Mazes

Simple maze representation and solvers.

See [The LeetCode example problem](https://leetcode.com/problems/trim-a-binary-search-tree/description/)

In [39]:

debugging = False
debugging = True
debugging2 = False

logging = True

def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))

def dbg2(f, *args):
    if debugging2:
        print(('  DBG:' + f).format(*args))

def log(f, *args):
    if logging:
        print((f).format(*args))
        
def logError(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))
        
def className(instance):
    return type(instance).__name__

In [40]:
from collections import namedtuple
Point = namedtuple('Point', ['row', 'col']) #, verbose=True)

In [41]:
import math
def pdiff(p1, p2):
    return Point(p1.row - p2.row, p1.col - p2.col)

def pdistance(p1, p2):
    return math.sqrt((p1.row - p2.row)**2 + (p1.col - p2.col)**2)


In [42]:
#--------------------------------------------------------------------------
class MazeMove(object):
    """ A single possible move """
    
    def __init__(self, fromCell, toCell):
        self.gValue = 0 # The known distance from the start
        self.hValue = 0 # An estimated heuristic to the solution
        self.cost = 0
        self.fromCell = fromCell
        self.toCell = toCell
        
    def calcCost(self, domain):
        """ Compute the cost of this move and store it """
        self.gValue = self.fromCell.gCost + 1 # Maze moves have fixed unit costs.
        self.hValue = domain.goalEstimate(self.toCell) # An estimated heuristic to the solution
        self.hValue = 0 # breadth first, trying least expensive paths in order.
        self.cost =  self.gValue + self.hValue
        
    def __repr__(self): # Used when displayed from a container.
        return "(MOVE@{2:.2f} {0}==>{1}".format(
            self.fromCell.pos, self.toCell.pos, self.gValue+self.hValue)
    
    def __str__(self): # Simple prints and single object formats.
        return "Move from:{0} to:{1}  (g={2},h={3:.2f})".format(
            self.fromCell, self.toCell, self.gValue, self.hValue)
    
    def __lt__(self, other): # Required for inclusion in priority queue
        if isinstance(other, self.__class__):
            return self.cost < other.cost    
        return NotImplemented
    
    def __eq__(self, other): # Optional when including in priority queue
        if isinstance(other, self.__class__):
            return self.cost == other.cost            
        return NotImplemented
    

In [43]:
class MazeCell(object):
    """ A single cell within the maze """

    def __init__(cell, pos):
        cell.blocked = False # True if this is a block.
        cell.gCost = 0 # Minimum cost to get to this position.
        cell.visited = 0
        cell.proposed = 0
        cell.bestPath = 0
        cell.pos = pos
        dbg2(" [{0},{1}],".format(pos.row, pos.col))

    def __str__(cell):
        return "Cell:{0}, visited={1} blocked={2}".format(cell.pos, cell.visited, cell.blocked)
    
    def __lt__(self, other): # Required for inclusion in priority queue
        if isinstance(other, self.__class__):
            return self.gCost < other.gCost    
        return NotImplemented
    
    def __eq__(self, other): # Optional when including in priority queue
        if isinstance(other, self.__class__):
            return self.gCost == other.gCost            
        return NotImplemented    

In [44]:
import heapq as hq

class BlockMaze(object):
    """ Representation of a maze with walls that take up entire cells """

    def __init__(self):
        self.maze = [] # A maze arranged as rows and columns of MazeCells.

    def loadMaze(self, mrows): 
        self.maze = []
        assert len(mrows) > 0 and isinstance(mrows, list)
        cols = 0
        for r,s in enumerate(mrows):
            row = []
            for c, cv in enumerate(s):
                cell = MazeCell(Point(r, c))
                cell.blocked = True if cv == 'X' else False
                row.append(cell)
            self.maze.append(row)
            # Require all rows to be the same length.
            if cols>0: assert len(row) == cols
            cols = len(row)
        self.rows = len(self.maze)
        self.cols = cols
            
    def printMaze(self):
        rc = len(self.maze)
        cc = len(self.maze[0])
        def pe(v):
            print(v,end='')
        def printTopBot():
            pe('+')
            for c in range(cc): pe('---')
            print('+')
        printTopBot()
        for row in self.maze:
            pe('|')
            for c in row:
                txt = '(X)' if c.blocked else '   '             
                if self.endCell and c.pos == self.endCell.pos: txt = "-!-"
                if c.proposed>0: txt = " . "
                if c.visited<0: txt = "-S-"
                if c.visited>0: txt = " * "
                #if c.bestPath: txt = '{' + txt[1] + '}'
                #if c.bestPath>1: txt = '+' + txt[1] + '+'
                if c.bestPath:   txt = ' @ '
                if c.bestPath>1: txt = ' + '
                pe(txt)
            print('|')
        printTopBot()

    def isValidPos(self, p):
        """ validates and adjusts cell position (or kills it to None) """
        row = p[0]
        col = p[1]
        return row >= 0 and row < self.rows and col >= 0 and col < self.cols
    
    def adjustedPos(self, p):
        """ validates and adjusts cell position (or kills it to None) """
        row = p.row
        col = p.col
        if row<0: row=self.rows+row           
        if col<0: col=self.cols+col
        if row >= 0 and row < self.rows and col >= 0 and col < self.cols:
            return Point(row, col)
        return None
        
    def getCell(self, p):
        cell = None
        if self.isValidPos(p):
            cell = self.maze[p.row][p.col]
        return cell
    
    @staticmethod                
    def up(p):
        return Point(p.row-1, p.col)

    @staticmethod                
    def down(p):
        return Point(p.row+1, p.col)

    @staticmethod                
    def right(p):
        return Point(p.row, p.col+1)

    @staticmethod     
    def left(p):
        return Point(p.row, p.col-1)
    
    @staticmethod   
    def allMoveMethods():
        return [__class__.up, __class__.down, __class__.left, __class__.right]
        
    def openMoves(self, pos):
        """ Returns a list of currently open moves from the position """
            
        def addOpenMove(here, p, moves):
            fromCell = self.getCell(here)
            toCell = self.getCell(p)
            if toCell:
                if toCell.visited != 0 or toCell.blocked:
                    toCell = None # Not a position open for moves.
            if toCell:
                move = MazeMove(fromCell, toCell)
                move.calcCost(self) # compute cost for move and guess remainder.
                moves.append(move)
        
        moves = []
        for f in __class__.allMoveMethods():
            addOpenMove(pos, f(pos),  moves)
        return moves
    
    def getMinTo(self, cell):
        """ Returns a list of cells with minimum cost to this one """
            
        def addMove(pos, moves):
            cell = self.getCell(pos)
            if cell: # Ignore moves that are outside of the maze or impossible
                #log('======= addMove pos={0} cell={1}', pos, cell)
                cost = cell.gCost 
                if cost:
                    hq.heappush(moves, (cost, cell))
        
        moves = []
        followedMoves = []
        pos = cell.pos
        for f in __class__.allMoveMethods():
            addMove(f(pos), followedMoves)
        minval = 0
        while True:
            try:
                m = hq.heappop(followedMoves)[1] # Get the cell from the tuple.
                if minval:
                    assert minval <= m.gCost
                    if minval < m.gCost: break # ignore non-minimum moves
                else:
                    minval = m.gCost # Select the minimum cost
            except IndexError:
                break            
            moves.append(m)
        return moves
        
    def showCurrentState(self):
        self.printMaze()
        
    def setStart(self, start):
        cell = self.getCell(self.adjustedPos(start))
        cell.visited = -1
        self.startCell = cell

    def setSuccess(self, end):
        cell = self.getCell(self.adjustedPos(end))
        self.endCell = cell

    def goalEstimate(self, cell):
        """ A safe acceptable heuristic, the straight line distance to the end cell """
        p1 = cell.pos
        p2 = self.endCell.pos
        return pdistance(p1, p2)
    
    def move(self, mv):
        """ Evaluate the move and mark the cell. True on success """
        c = mv.toCell
        if (c.visited):
            # The cost to get to this cell is its minimum cost.
            c.gCost = min(c.gCost, mv.fromCell.gCost+1)           
        else:
            c.gCost = mv.fromCell.gCost+1
        c.visited += 1
        return c.pos == self.endCell.pos
    
    def propose(self, mv):
        """ Mark the cell as proposed """
        c = mv.toCell
        c.proposed += 1
        
    def markBestPaths(self, toCell=None):
        """ Mark all best paths to solution. """
        c = self.endCell if toCell is None else toCell
        if c.pos != self.startCell.pos:
            best = self.getMinTo(c)
            for x in best:
                x.bestPath += 1
                # Follow the path unless it is a path junction.
                if x.bestPath == 1:
                    self.markBestPaths(x)       
            

In [45]:
import heapq as hq

class Solver(object):
    
    def __init__(self, domain):
        self.domain = domain # The environment in which a solution must be found.
    
    def solve(self, start, success):
        queue = [] # Start with an empty moves queue.
        d = self.domain
        d.setStart(start)
        d.setSuccess(success)
        d.showCurrentState()
        
        for mv in d.openMoves(start):
            hq.heappush(queue, mv)
        
        maxmoves = 30
        move = hq.heappop(queue)
        moveno = 0
        while moveno < maxmoves and not d.move(move):
            # Look for some new moves
            for mv in d.openMoves(move.toCell.pos):
                hq.heappush(queue, mv) # Push 0 or more moves
            while True: # Proposed moves may be invalidated by other paths.
                try:
                    move = None
                    move = hq.heappop(queue)
                except IndexError:
                    log('- NO SOLUTION IS POSSIBLE!')
                    break
                if move.toCell.visited == 0: break
            moveno += 1
            if move:
                d.propose(move)
                print('---{0}---', moveno, move)
                d.showCurrentState()
            else:
                moveno = maxmoves
        
        if moveno >= maxmoves:
            log('- LIFE SUCKS!')
        else:
            log('*************** WHOOOOPIEEE *******************')
        d.markBestPaths()
        d.showCurrentState()        
        return queue
    

In [46]:
x = BlockMaze()

x.loadMaze(["   X    ",
            "        ",
            " X    X ",
            "   X    ",
            "     X  "])

x.loadMaze(["   X    ",
            "   X XX ",
            " X    X ",
            "   X XX ",
            " X   X  "])

x.loadMaze(["        ",
            "  XX    ",
            " X   XX ",
            " X XX   ",
            "        "])


#x.printMaze()

s=Solver(x)
s.solve(Point(0,0), Point(-1,-1))


+------------------------+
|-S-                     |
|      (X)(X)            |
|   (X)         (X)(X)   |
|   (X)   (X)(X)         |
|                     -!-|
+------------------------+
---{0}--- 1 Move from:Cell:Point(row=0, col=0), visited=-1 blocked=False to:Cell:Point(row=0, col=1), visited=0 blocked=False  (g=1,h=0.00)
+------------------------+
|-S- .                   |
| *    (X)(X)            |
|   (X)         (X)(X)   |
|   (X)   (X)(X)         |
|                     -!-|
+------------------------+
---{0}--- 2 Move from:Cell:Point(row=1, col=0), visited=1 blocked=False to:Cell:Point(row=2, col=0), visited=0 blocked=False  (g=2,h=0.00)
+------------------------+
|-S- *                   |
| *    (X)(X)            |
| . (X)         (X)(X)   |
|   (X)   (X)(X)         |
|                     -!-|
+------------------------+
---{0}--- 3 Move from:Cell:Point(row=0, col=1), visited=1 blocked=False to:Cell:Point(row=1, col=1), visited=0 blocked=False  (g=2,h=0.00)
+--------------

[(MOVE@11.00 Point(row=3, col=5)==>Point(row=3, col=6),
 (MOVE@11.00 Point(row=3, col=7)==>Point(row=3, col=6),
 (MOVE@11.00 Point(row=4, col=6)==>Point(row=4, col=7),
 (MOVE@11.00 Point(row=4, col=6)==>Point(row=3, col=6)]

In [8]:
import pprint as pp
pdistance([1,1], [2,2])
[1, 1] == [1, 1]
print(x.maze[0][0])

Cell:[0, 0], visited=-1 blocked=False


In [9]:
import heapq as hq
h = []
hq.heappush(h, (5, 'write code'))
hq.heappush(h, (7, 'release product'))
hq.heappush(h, (1, 'write spec'))
hq.heappush(h, (3, 'create tests'))
hq.heappop(h)
(1, 'write spec')

(1, 'write spec')

In [10]:
(1, 2, 3, 4)[2]

3